In [ ]:
import os
import numpy as np
import requests
from zipfile import ZipFile
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
from itertools import product
import time
!pip install kaggle
!pip3 install ann_visualizer
!pip install graphviz
from ann_visualizer.visualize import ann_viz
from graphviz import Source

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification
!unzip muffin-vs-chihuahua-image-classification.zip -d /content/dataset

In [ ]:
def load_dataset():
    x = []
    y = []
    classes = ['chihuahua', 'muffin']
    dataset_path = '/content/dataset'

    # Load training images and labels
    for class_name in classes:
        class_path = os.path.join(dataset_path, 'train', class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (32, 32))
            x.append(img)
            y.append(classes.index(class_name))

    # Load testing images and labels
    for class_name in classes:
        class_path = os.path.join(dataset_path, 'test', class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (32, 32))
            x.append(img)
            y.append(classes.index(class_name))

    x = np.array(x)
    y = np.array(y)

    return x,y

In [ ]:
# Load the dataset
x,y = load_dataset()


In [ ]:
def plot_training_history(history, learning_rate, batch_size, epochs, model):
    # Plot the training loss and accuracy on the same plot
    epochs = range(1, len(history.history['loss']) + 1)
    plt.plot(epochs, history.history['loss'], 'b',label='Training Loss')
    plt.plot(epochs, history.history['accuracy'], 'r', label='Training Accuracy')
    plt.plot(epochs, [1 - accuracy for accuracy in history.history['val_accuracy']], label='Validation zero - one loss', marker='o', linestyle='-', color='black')

    plt.title('Training and Validation Loss and Accuracy')
    plt.xlabel('Epoch')
    plt.suptitle(f"Number of epochs: { epochs}  Learning rate:  {learning_rate}  Batch size  {batch_size} {model}", ha='center')
    plt.legend()
    plt.show()

In [ ]:
def create_model1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    ann_viz(model, view=True, title="Neural network plot")
    Source.from_file('/content/network.gv')
    return model

In [ ]:
def create_model2():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    ann_viz(model, view=True, title="Neural network plot")
    Source.from_file('/content/network.gv')
    return model


In [ ]:
def create_model3():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    ann_viz(model, view=True, title="Neural network plot")
    Source.from_file('/content/network.gv')
    return model

In [ ]:
n_splits = 5

hyperparameters = {
    'learning_rate': [0.0001],
    'batch_size': [128],
    'epochs': [8,16,24],
}

results = []
# Define the K-Fold cross-validation
kfold = KFold(n_splits=5)
fold = 1
cv_accuracy = []
cv_loss = []
cv_zero_one_loss = []
learning_times = []

for hyperparam in product(*hyperparameters.values()):
    hyperparam = dict(zip(hyperparameters.keys(), hyperparam))
    learning_rate = hyperparam['learning_rate']
    batch_size = hyperparam['batch_size']
    epochs = hyperparam['epochs']
    fold = 1
    for train_index, validation_index in kfold.split(x, y):
        # Split the data into training and validation sets
        x_train_fold, x_validation_fold = x[train_index], x[validation_index]
        y_train_fold, y_validation_fold = y[train_index], y[validation_index]

        print(f"Fold: {fold}")
        print("x_train_fold size: ", len(x_train_fold))
        print("x_validation_fold size: ", len(x_validation_fold))
        print("y_train_fold size: ", len(y_train_fold))
        print("y_validation_fold size: ", len(y_validation_fold))

        # Create the model
        model = create_model2()
        model.summary()

        # Compile the model
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss=binary_crossentropy, metrics=['accuracy'])

        # Train the model
        start = time.time()
        history = model.fit(x_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_validation_fold, y_validation_fold))
        end = time.time()
        learning_times.append(end - start) # time in seconds

        plot_training_history(history, learning_rate, batch_size, epochs, "model2")

        validation_loss, validation_accuracy = model.evaluate(x_validation_fold, y_validation_fold, verbose=0)
        validation_zero_one_loss = 1 - validation_accuracy

        cv_accuracy.append(validation_accuracy)
        cv_loss.append(validation_loss)
        cv_zero_one_loss.append(validation_zero_one_loss)

        print("test accuracy:", validation_accuracy)
        print("test loss:", validation_loss)
        print("Zero-One Loss on Test Fold: ",validation_zero_one_loss )

        #get_metrics_result
        print("get_metrics_result: ", model.get_metrics_result())
        # Store the results
        results.append({
                  'hyperparameters': hyperparam,
                  'val_loss': validation_loss,
                  'val_acc': validation_accuracy,
              })
        fold += 1

    # Compute the average cross-validated accuracy
    mean_cv_accuracy = np.mean(cv_accuracy)
    mean_cv_loss = np.mean(cv_loss)
    mean_cv_zero_one_loss = np.mean(cv_zero_one_loss)
    mean_learning_time = np.mean(learning_times)
    print("Cross-Validated Accuracy:", mean_cv_accuracy)
    print("Cross-Validated Loss:", mean_cv_loss)
    print("mean_cv_zero_one_loss", mean_cv_zero_one_loss)
    print("cv_zero_one_loss", cv_zero_one_loss)
    print("mean_learning_time",mean_learning_time )
    print("learning times",learning_times )
    cv_accuracy.clear()
    cv_loss.clear()
    cv_zero_one_loss.clear()
    learning_times.clear()
print(results)